# Models for ozone 

In this notebook, we develop the models for the transformed dataset considering the exogenous variable to be the **Ozone**.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import os
import tqdm
from utilities import Utilities

# plotting
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

# models 
from sklearn.linear_model import ElasticNetCV, ElasticNet, LinearRegression
import statsmodels.api as sm
from sklearn.svm import LinearSVR, SVR
from sklearn.ensemble import RandomForestRegressor

# others
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, PowerTransformer, PolynomialFeatures

from scipy.stats import jarque_bera

# R packages
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import FloatVector, r

norm = importr("norm")

## Definitions for the notebook

In [4]:
# For local folder
IMAGES_FOLDER = "../notes/images/"
# For Google colab
#IMAGES_FOLDER = "drive/MyDrive/Arquivos Acadêmicos/Disciplinas FGV/Machine Learning/images/"

# For local folder
location = "../data/"
# For Google colab
#location = "drive/MyDrive/Arquivos Acadêmicos/Disciplinas FGV/Machine Learning/"

sns.set()

pd.set_option('precision', 3)
pd.options.mode.chained_assignment = None

utility = Utilities()

%matplotlib inline

## Importing the dataset

In [5]:
air_data = pd.read_csv(location + "RiodeJaneiro_MonitorAr_hourly_p3.csv", index_col = 0)
air_data.weekend = air_data.weekend.astype(int)
air_data = air_data.reset_index().drop(columns="index")
air_data.head()

,year,month,day,CodNum,Lat,Lon,Chuva,Pres,RS,Temp,...,CO_lag24,CO_MA24,O3_lag1,O3_lag2,O3_lag24,O3_MA24,PM10_lag1,PM10_lag2,PM10_lag24,PM10_MA24
0,2011,1,2,1,-22.965,-43.180,3.617,-1.530,-1.128,-0.140,...,-0.763,-0.936,-0.738,-0.543,-1.190,-0.365,0.101,0.078,0.087,0.082
1,2011,1,2,7,-22.898,-43.222,-0.272,-1.513,-0.565,0.560,...,-1.678,-1.232,-0.612,-0.997,-0.724,-0.187,-0.307,-0.967,-1.858,-1.136
2,2011,1,2,3,-22.908,-43.178,3.617,-1.557,-0.720,-0.470,...,-0.634,-0.166,-0.211,-0.717,-0.169,-0.027,-0.677,-0.677,-1.617,-1.141
3,2011,1,2,8,-22.925,-43.233,3.617,-1.909,-0.194,-1.068,...,-0.897,-1.283,-0.267,-0.184,-1.028,0.274,-0.773,-0.583,0.495,-1.061
4,2011,1,2,7,-22.898,-43.222,3.617,-1.658,-0.567,0.548,...,-1.591,-1.206,-1.300,-0.612,-1.099,-0.204,-0.484,-0.307,-0.967,-1.136


## Inverse Power Transformation 

Just for future transformations on ozone data. 

In [6]:
air_data_ = pd.read_csv(location + "RiodeJaneiro_MonitorAr_hourly_p2.csv", index_col = 0)

gases = ['CO', 'O3', 'PM10']

pt_gases = {key: PowerTransformer(method = 'yeo-johnson', 
                                  standardize=True).fit(air_data_[[key]]) for key in gases}

del air_data_

## Train and Test

In [7]:
df_train = air_data[air_data.train].drop(columns='train')
df_test = air_data[~air_data.train].drop(columns='train')

x_train = df_train.drop(columns=["O3", 'CO', 'PM10', 'aiq', 'Lat', 'Lon'])
x_test = df_test.drop(columns=["O3", 'CO', 'PM10', 'aiq', 'Lat', 'Lon'])

## Models for each gas and each station 

Now we are going to apply the models for each of the best models. 

In [9]:
poly = PolynomialFeatures(2)

best_features_o3 = ['year O3_lag1', 'RS', 'RS hour_cos', 'year Vel_Vento', 
                    'O3_lag2', 'year O3_MA24', 'hour_cos PM10_lag1', 'RS Temp', 
                    'RS O3_lag2', 'O3_lag1^2']
best_features_co = ['CO_lag1', 'hour_sin^2', 'CO_lag24', 'hour_cos', 'year Vel_Vento', 
                    'CO_MA24', 'RS PM10_lag1', 'CO_lag2', 'Vel_Vento^2']
best_features_pm10 = ['PM10_lag1', 'year PM10_MA24', 'hour_cos', 'O3_lag1', 
                      'year O3_lag2', 'hour_sin hour_cos', 'year Temp', 'RS O3_lag2', 'O3_lag1^2']

## Ozone models 

For the ozone, we test SVR with kernels RBF/linear and best features.

### SVR with RBF Kernel and best features

In [10]:
svr = make_pipeline(StandardScaler(), 
                    SVR(kernel = 'rbf', 
                        degree = 2, 
                        epsilon = 0.2, 
                        C = 1.0,
                        max_iter = 200000))

In [12]:
for station in tqdm.tqdm(range(1,9)): 

    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_o3 = df_train[df_train.CodNum == station].O3
    y_test_o3 = df_test[df_test.CodNum == station].O3
    
    x_train_SP_poly = pd.DataFrame(poly.fit_transform(x_train_SP), 
                                   columns = poly.get_feature_names(x_train_SP.columns), 
                                   index = x_train_SP.index)
    x_test_SP_poly = pd.DataFrame(poly.fit_transform(x_test_SP), 
                                  columns = poly.get_feature_names(x_test_SP.columns),
                                  index = x_test_SP.index)

    svr.fit(x_train_SP_poly[best_features_o3], y_train_o3)
    
    y_pred = svr.predict(x_test_SP_poly[best_features_o3])

    y_train_pred = svr.predict(x_train_SP_poly[best_features_o3])

    r2_train = r2_score(y_train_o3, y_train_pred)
    r2_test  = r2_score(y_test_o3, y_pred)
    mae_train = mean_absolute_error(y_train_o3, y_train_pred)
    mae_test  = mean_absolute_error(y_test_o3, y_pred)
    rmse_train = mean_squared_error(y_train_o3, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_o3, y_pred, squared = False)

    utility.save_metrics("O3", station, "svr_rbf_best_features", 
                         {'epsilon': 0.2, 'C': 1.0},    
                          r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

100%|██████████| 8/8 [23:36<00:00, 177.11s/it]


### SVR with Linear Kernel and best features

In [14]:
svr_linear = make_pipeline(StandardScaler(), 
                           LinearSVR(epsilon = 0.2, 
                                     C = 1.0,
                                     max_iter = 200000))

In [19]:
for station in tqdm.tqdm(range(1,9)): 

    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_o3 = df_train[df_train.CodNum == station].O3
    y_test_o3 = df_test[df_test.CodNum == station].O3
    
    x_train_SP_poly = pd.DataFrame(poly.fit_transform(x_train_SP), 
                                   columns = poly.get_feature_names(x_train_SP.columns), 
                                   index = x_train_SP.index)
    x_test_SP_poly = pd.DataFrame(poly.fit_transform(x_test_SP), 
                                  columns = poly.get_feature_names(x_test_SP.columns),
                                  index = x_test_SP.index)

    svr_linear.fit(x_train_SP_poly[best_features_o3], y_train_o3)
    
    y_pred = svr_linear.predict(x_test_SP_poly[best_features_o3])

    y_train_pred = svr_linear.predict(x_train_SP_poly[best_features_o3])

    r2_train = r2_score(y_train_o3, y_train_pred)
    r2_test  = r2_score(y_test_o3, y_pred)
    mae_train = mean_absolute_error(y_train_o3, y_train_pred)
    mae_test  = mean_absolute_error(y_test_o3, y_pred)
    rmse_train = mean_squared_error(y_train_o3, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_o3, y_pred, squared = False)

    utility.save_metrics("O3", station, "svr_linear_best_features", 
                         {'epsilon': 0.2, 'C': 1.0},    
                          r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

100%|██████████| 8/8 [00:44<00:00,  5.61s/it]


### Linear regression with EM and best features

In [37]:
for station in tqdm.tqdm(range(1,9)):
        
    # Dataset with imputed data
    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_o3 = df_train[df_train.CodNum == station].O3
    y_test_o3 = df_test[df_test.CodNum == station].O3
    
    x_train_SP_poly = pd.DataFrame(poly.fit_transform(x_train_SP), 
                                   columns = poly.get_feature_names(x_train_SP.columns), 
                                   index = x_train_SP.index)
    x_test_SP_poly = pd.DataFrame(poly.fit_transform(x_test_SP), 
                                  columns = poly.get_feature_names(x_test_SP.columns),
                                  index = x_test_SP.index)

    # Dataset with missing values
    df = utility.linear_regression_em_preparation(location, gas_name='O3', loc = station)
    df_2 = pd.DataFrame()
    # Adding polynomial and interactions terms
    for feat in best_features_o3: 
        feats = feat.split()
        if len(feats) == 2: 
            df_2[feat] = df[feats[0]]*df_train[feats[1]]
        elif feats[0][-2] == '^': 
            df_2[feat] = df[feats[0][:-2]]**2
        else: 
            df_2[feat] = df[feats[0]]
    df_2['O3'] = df['O3']

    X_r = FloatVector(df_2.values.flatten())
    m = r['matrix'](X_r, ncol = df_2.shape[1], byrow = True)

    s = norm.prelim_norm(m)  
    theta = norm.em_norm(s)
    params = norm.getparam_norm(s,theta,corr=False)
    params = dict(zip(params.names, map(np.array,list(params))))

    mu_y = params['mu'][-1]
    mu_X = params['mu'][:-1].reshape(-1,1)

    Sigma_XX = params['sigma'][:-1,:-1]
    Sigma_yX = params['sigma'][-1,:-1].reshape(1,-1)
    Sigma_Xy = params['sigma'][:-1,-1].reshape(-1,1)
    Sigma_yy = params['sigma'][-1,-1]

    inv_Sigma_XX = np.linalg.inv(Sigma_XX)

    beta = np.hstack([mu_y - Sigma_yX@inv_Sigma_XX@mu_X, Sigma_yX@inv_Sigma_XX]).reshape(-1,1)

    y_pred = x_test_SP_poly[best_features_o3]@beta[1:] + beta[0]

    y_train_pred = x_train_SP_poly[best_features_o3]@beta[1:] + beta[0]

    r2_train = r2_score(y_train_o3, y_train_pred)
    r2_test  = r2_score(y_test_o3, y_pred)
    mae_train = mean_absolute_error(y_train_o3, y_train_pred)
    mae_test  = mean_absolute_error(y_test_o3, y_pred)
    rmse_train = mean_squared_error(y_train_o3, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_o3, y_pred, squared = False)

    utility.save_metrics("O3", station, "linear_regression_em_best_features", 
                         {},    
                         r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

  0%|          | 0/8 [00:00<?, ?it/s]

Iterations of EM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...


 12%|█▎        | 1/8 [00:15<01:47, 15.33s/it]

Iterations of EM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...58...59...60...61...


 25%|██▌       | 2/8 [00:30<01:29, 14.97s/it]

...

 75%|███████▌  | 6/8 [01:27<00:28, 14.38s/it]

Iterations of EM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...


 88%|████████▊ | 7/8 [01:41<00:14, 14.18s/it]

Iterations of EM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...58...59...60...61...62...63...64...65...66...67...68...69...70...71...72...73...74...75...76...77...78...79...80...81...82...83...84...85...86...87...88...89...90...91...92...93...94...95...96...97...98...99...100...101...102...103...104...105...106...107...108...109...110...111...112...113...114...115...116...117...118...119...120...121...122...123...124...125...126...127...128...129...130...131...132...133...134...135...136...137...138...139...140...141...142...143...144...145...146...147...148...149...150...151...152...153...154...155...156...157...158...159...160...161...162...163...164...165...166...167...168...169...170...171...172...173...174...175...176...177...178...179...180...181...182

100%|██████████| 8/8 [02:32<00:00, 19.11s/it]


## CO Models

For the carbon monoxide, we test Random Forest and SVR with linear kernel, both with best features. Pedra da Guaratiba (6) does not measure this quantity.

### Random Forest with best features

In [20]:
rand_forest = RandomForestRegressor(n_estimators = 500, 
                                    criterion = 'mse', 
                                    min_samples_split = 5, 
                                    max_features = 'sqrt',
                                    ccp_alpha = 0.0,
                                    n_jobs = 2,
                                    )

In [21]:
for station in tqdm.tqdm(range(1,9)): 
    
    if station == 6: continue

    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_co = df_train[df_train.CodNum == station].CO
    y_test_co = df_test[df_test.CodNum == station].CO
    
    x_train_SP_poly = pd.DataFrame(poly.fit_transform(x_train_SP), 
                                   columns = poly.get_feature_names(x_train_SP.columns), 
                                   index = x_train_SP.index)
    x_test_SP_poly = pd.DataFrame(poly.fit_transform(x_test_SP), 
                                  columns = poly.get_feature_names(x_test_SP.columns),
                                  index = x_test_SP.index)

    rand_forest.fit(x_train_SP_poly[best_features_co], y_train_co)
    
    y_pred = rand_forest.predict(x_test_SP_poly[best_features_co])

    y_train_pred = rand_forest.predict(x_train_SP_poly[best_features_co])

    r2_train = r2_score(y_train_co, y_train_pred)
    r2_test  = r2_score(y_test_co, y_pred)
    mae_train = mean_absolute_error(y_train_co, y_train_pred)
    mae_test  = mean_absolute_error(y_test_co, y_pred)
    rmse_train = mean_squared_error(y_train_co, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_co, y_pred, squared = False)

    utility.save_metrics("CO", station, "random_forest_best_features", 
                         {'s': 5, 'c': 0, 'B': 500},    
                         r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

100%|██████████| 8/8 [03:50<00:00, 28.78s/it]


### SVR with linear kernel and best features

In [22]:
svr = make_pipeline(StandardScaler(), 
                    LinearSVR(epsilon = 0.2, 
                              C = 0.01,
                              max_iter = 10000))

In [23]:
for station in tqdm.tqdm(range(1,9)): 
    
    if station == 6: continue

    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_co = df_train[df_train.CodNum == station].CO
    y_test_co = df_test[df_test.CodNum == station].CO
    
    x_train_SP_poly = pd.DataFrame(poly.fit_transform(x_train_SP), 
                                   columns = poly.get_feature_names(x_train_SP.columns), 
                                   index = x_train_SP.index)
    x_test_SP_poly = pd.DataFrame(poly.fit_transform(x_test_SP), 
                                  columns = poly.get_feature_names(x_test_SP.columns),
                                  index = x_test_SP.index)

    svr.fit(x_train_SP_poly[best_features_co], y_train_co)
    
    y_pred = svr.predict(x_test_SP_poly[best_features_co])

    y_train_pred = svr.predict(x_train_SP_poly[best_features_co])

    r2_train = r2_score(y_train_co, y_train_pred)
    r2_test  = r2_score(y_test_co, y_pred)
    mae_train = mean_absolute_error(y_train_co, y_train_pred)
    mae_test  = mean_absolute_error(y_test_co, y_pred)
    rmse_train = mean_squared_error(y_train_co, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_co, y_pred, squared = False)

    utility.save_metrics("CO", station, "svr_linear_best_features", 
                         {'epsilon': 0.2, 'C': 0.01},    
                         r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

100%|██████████| 8/8 [00:02<00:00,  2.80it/s]


### Linear regression with EM imputation and best features

In [36]:
for station in tqdm.tqdm(range(1,9)):
    
    if station == 6: continue
    
    # Dataset with imputed data
    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_co = df_train[df_train.CodNum == station].CO
    y_test_co = df_test[df_test.CodNum == station].CO
    
    x_train_SP_poly = pd.DataFrame(poly.fit_transform(x_train_SP), 
                                   columns = poly.get_feature_names(x_train_SP.columns), 
                                   index = x_train_SP.index)
    x_test_SP_poly = pd.DataFrame(poly.fit_transform(x_test_SP), 
                                  columns = poly.get_feature_names(x_test_SP.columns),
                                  index = x_test_SP.index)

    # Dataset with missing values
    df = utility.linear_regression_em_preparation(location, gas_name='CO', loc = station)
    df_2 = pd.DataFrame()
    # Adding polynomial and interactions terms
    for feat in best_features_co: 
        feats = feat.split()
        if len(feats) == 2: 
            df_2[feat] = df[feats[0]]*df_train[feats[1]]
        elif feats[0][-2] == '^': 
            df_2[feat] = df[feats[0][:-2]]**2
        else: 
            df_2[feat] = df[feats[0]]
    df_2['CO'] = df['CO']

    X_r = FloatVector(df_2.values.flatten())
    m = r['matrix'](X_r, ncol = df_2.shape[1], byrow = True)

    s = norm.prelim_norm(m)  
    theta = norm.em_norm(s)
    params = norm.getparam_norm(s,theta,corr=False)
    params = dict(zip(params.names, map(np.array,list(params))))

    mu_y = params['mu'][-1]
    mu_X = params['mu'][:-1].reshape(-1,1)

    Sigma_XX = params['sigma'][:-1,:-1]
    Sigma_yX = params['sigma'][-1,:-1].reshape(1,-1)
    Sigma_Xy = params['sigma'][:-1,-1].reshape(-1,1)
    Sigma_yy = params['sigma'][-1,-1]

    inv_Sigma_XX = np.linalg.inv(Sigma_XX)

    beta = np.hstack([mu_y - Sigma_yX@inv_Sigma_XX@mu_X, Sigma_yX@inv_Sigma_XX]).reshape(-1,1)

    y_pred = x_test_SP_poly[best_features_co]@beta[1:] + beta[0]

    y_train_pred = x_train_SP_poly[best_features_co]@beta[1:] + beta[0]

    r2_train = r2_score(y_train_co, y_train_pred)
    r2_test  = r2_score(y_test_co, y_pred)
    mae_train = mean_absolute_error(y_train_co, y_train_pred)
    mae_test  = mean_absolute_error(y_test_co, y_pred)
    rmse_train = mean_squared_error(y_train_co, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_co, y_pred, squared = False)

    utility.save_metrics("CO", station, "linear_regression_em_best_features", 
                         {},    
                         r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

  0%|          | 0/8 [00:00<?, ?it/s]

Iterations of EM: 
1...2...3...4...5...6...7...


 12%|█▎        | 1/8 [00:09<01:07,  9.63s/it]

Iterations of EM: 
1...2...3...4...5...6...7...8...9...


 25%|██▌       | 2/8 [00:20<01:01, 10.29s/it]

...

 62%|██████▎   | 5/8 [00:51<00:30, 10.24s/it]

Iterations of EM: 
1...2...3...4...5...6...7...8...


 88%|████████▊ | 7/8 [01:01<00:07,  7.53s/it]

Iterations of EM: 
1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...58...59...60...61...62...


100%|██████████| 8/8 [01:11<00:00,  8.91s/it]


## PM10 Models

For the particulate matter, we test Random Forest and SVR with linear kernel. 

### Random Forest

In [24]:
rand_forest = RandomForestRegressor(n_estimators = 500, 
                                    criterion = 'mse', 
                                    min_samples_split = 10, 
                                    max_features = 'sqrt',
                                    ccp_alpha = 0.0,
                                    n_jobs = 2,
                                    )

In [25]:
for station in tqdm.tqdm(range(1,9)): 

    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_pm10 = df_train[df_train.CodNum == station].PM10
    y_test_pm10 = df_test[df_test.CodNum == station].PM10

    rand_forest.fit(x_train_SP, y_train_pm10)
    
    y_pred = rand_forest.predict(x_test_SP)

    y_train_pred = rand_forest.predict(x_train_SP)

    r2_train = r2_score(y_train_pm10, y_train_pred)
    r2_test  = r2_score(y_test_pm10, y_pred)
    mae_train = mean_absolute_error(y_train_pm10, y_train_pred)
    mae_test  = mean_absolute_error(y_test_pm10, y_pred)
    rmse_train = mean_squared_error(y_train_pm10, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_pm10, y_pred, squared = False)

    utility.save_metrics("PM10", station, "random_forest", 
                         {'s': 10, 'c': 0.0, 'B': 100},    
                         r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

100%|██████████| 8/8 [06:33<00:00, 49.22s/it]


### SVR Linear

In [26]:
svr = make_pipeline(StandardScaler(), 
                    LinearSVR(epsilon = 0.01, 
                              C = 0.1,
                              max_iter = 10000))

In [27]:
for station in tqdm.tqdm(range(1,9)): 

    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_pm10 = df_train[df_train.CodNum == station].PM10
    y_test_pm10 = df_test[df_test.CodNum == station].PM10

    svr.fit(x_train_SP, y_train_pm10)
    
    y_pred = svr.predict(x_test_SP)

    y_train_pred = svr.predict(x_train_SP)

    r2_train = r2_score(y_train_pm10, y_train_pred)
    r2_test  = r2_score(y_test_pm10, y_pred)
    mae_train = mean_absolute_error(y_train_pm10, y_train_pred)
    mae_test  = mean_absolute_error(y_test_pm10, y_pred)
    rmse_train = mean_squared_error(y_train_pm10, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_pm10, y_pred, squared = False)

    utility.save_metrics("PM10", station, "svr_linear", 
                         {'epsilon': 0.01, 'C': 0.1},    
                         r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

100%|██████████| 8/8 [00:11<00:00,  1.39s/it]


### SVR with RBF Kernel and best features

In [29]:
svr = make_pipeline(StandardScaler(), 
                    SVR(kernel = 'rbf', 
                        degree = 2, 
                        epsilon = 0.01, 
                        C = 0.1,
                        max_iter = 200000))

In [30]:
for station in tqdm.tqdm(range(1,9)): 

    x_train_SP = x_train[x_train.CodNum == station].drop(columns="CodNum")
    x_test_SP = x_test[x_test.CodNum == station].drop(columns="CodNum")

    y_train_pm10 = df_train[df_train.CodNum == station].PM10
    y_test_pm10 = df_test[df_test.CodNum == station].PM10
    
    x_train_SP_poly = pd.DataFrame(poly.fit_transform(x_train_SP), 
                                   columns = poly.get_feature_names(x_train_SP.columns), 
                                   index = x_train_SP.index)
    x_test_SP_poly = pd.DataFrame(poly.fit_transform(x_test_SP), 
                                  columns = poly.get_feature_names(x_test_SP.columns),
                                  index = x_test_SP.index)

    svr.fit(x_train_SP_poly[best_features_pm10], y_train_pm10)
    
    y_pred = svr.predict(x_test_SP_poly[best_features_pm10])

    y_train_pred = svr.predict(x_train_SP_poly[best_features_pm10])

    r2_train = r2_score(y_train_pm10, y_train_pred)
    r2_test  = r2_score(y_test_pm10, y_pred)
    mae_train = mean_absolute_error(y_train_pm10, y_train_pred)
    mae_test  = mean_absolute_error(y_test_pm10, y_pred)
    rmse_train = mean_squared_error(y_train_pm10, y_train_pred, squared = False)
    rmse_test  = mean_squared_error(y_test_pm10, y_pred, squared = False)

    utility.save_metrics("PM10", station, "svr_rbf_best_features", 
                         {'epsilon': 0.01, 'C': 0.1},    
                         r2_train, r2_test, mae_train, mae_test, rmse_train, rmse_test)

100%|██████████| 8/8 [43:09<00:00, 323.66s/it]


## Table aggregating the results

In [38]:
import json 

with open('../data/models.json', 'r') as f: 
    models = json.load(f)

In [41]:
df_models = pd.DataFrame()
for i in models.keys():
    df_models = df_models.append(pd.json_normalize(models[i]), ignore_index = True)